# 04 - Calculate the probabilites from the $\chi^2$ files

:::{warning}
Work in progress
:::

We now explain how to use a wrapper function to make all of the probability calculations necessary for a given star. We also show a wrapper function that will generate a PDF with useful diagnostic graphs. 

You will need to import the packages below:

In [1]:
import pyRaven as rav
import numpy as np

## 1. The calculation itself

The commands are relatively simple. 

1. We load up the <a href="../API/params.html#pyRaven.params.parameters">`Parameters`</a> object and the <a href="../API/data.html#pyRaven.data.DataPacket">`DataPacket`</a> object that were used to calculate the $\chi^2$ values on our grid of parameters.
2. We use the <a href="../API/BayesObjects.html#pyRaven.BayesObjects.combine_obs">`combine_obs`</a> wrapper function to get all of the posterior probabilities. 
    :::{note} 
    For the parameter estimation, `param['grid']['noise_grid']` must be defined in the <a href="../API/params.html#pyRaven.params.parameters">`Parameters`</a> object. We recommend a grid between 0.1 and 2.0 with 0.1 intervals. 
    
    While we are at it, we will also include the `Jeff_B` parameter that sets the break in the modified Jeffrey's prior (i.e. the value of Bpole below which the prior becomes flat -- here we use 66.9 G). This is not strictly needed here, but we will need it later. 
    :::




In [2]:
# Read in the param structure that was used to create the chi square data
# and the datapacket that was used. 
param = rav.params.read_parameters('ExampleOutput/03-ChiSquareCalcLoop/param_loop.json')
datapacket = rav.data.read_packet('ExampleOutput/01-DataSetup/ExamplePacket.h5')

In [3]:
# Calculate the likelihood for the parameter estimation calculations
# ** For this you will need a grid for the noise scale parameter **
param['grid']['noise_grid'] = np.arange(0.1,2.1,0.1)
param['grid']['Jeff_B'] = 66.9
# Write it back to file, so that we have a record of it later. 
param.write('ExampleOutput/04-CalculateProbabilities/param_prob.json')
chi_folder = 'ExampleOutput/03-ChiSquareCalcLoop/LoopOutput'

rav.BayesObjects.calc_all_LHs(param, datapacket, chi_folder, output_path='ExampleOutput/04-CalculateProbabilities')


The goal of this wrapper is to make calculations and create associated h5 files with the output data. 

If needed, all of these files can be examined afterward by loading them up with the appropriate object class in the <a href="../API/BayesObjects.html">`BayesObjects`</a> module, which also provide class functions to manipulate (and graph) this data, if some custom analysis is needed. 
See LINK TO NOTEBOOK for details. 

## 2. So, what are all of those files?

To explain the outputs of the wrapper function, we need to quickly recap how the bayesian analysis works:

>We have **two competitive hypothesis**: 
>
>* M_1: The star is magnetic, 
>* M_0: the star is not magnetic.
>
>We have a model for the "*the star is magnetic*", which has some parameters $\vec{\theta}$ ($B_\mathrm{pole}$, $\beta$, $i$, $\phi$). The 'model' for the "*star is not magnetic*" is simply that Stokes V = 0, which has no model parameter. 
>
>A few Bayesian definitions:
>
>* **A hypothesis comparison**: we compute the *odds ratio* between two competing hypothesis ("*the star is magnetic*" versus "*the star is not magnetic*")
>* **A parameter estimation**: we compute the probability of the model parameters assuming that the associated hypothesis is true. 
>
>A few more definitions:
>
>* The **prior** is the probabilily of a given hypothesis/parameter in the absence of new data. 
>* The **likelihood** is the probability of getting the current dataset if the hypothesis/parameters are the true hypothesis/parameters. This is closely related to (and calculated from) the $\chi^2$. 
>* The **posterior** is the probability of a hypothesis/parameter while taking into account both the prior and the likelihood. 
>
>And a few more definitions:
>
>* The output of a bayesian parameter estimation is not a probability but a **probability density**. Therefore to know the probability that the real value of parameter $x$ is between $x_1$ and $x_2$, one must integrate the probability density between $x_1$ and $x_2$. 
>
>* **Normalization**: the probability of a model over the whole set of parameters considered must be 1.0 (if the hypothesis is true, and the model is right, then the probability that the real value of the parameters is in the range covered by the parameters must be 1.0)
>* **Marginalization**: Imagine that we have a model with two parameters $\theta_1$ and $\theta_2$. The bayesian calculation gives us the joint posterior probability density $p(\theta_1, \theta_2)$. But let's say that we are interested only in the probability density with respect to $\theta_1$. In order words, what is the probability density for a particular value of $\theta_1$ with $\theta_2$ being able to be anything. Mathematically, this will be $p(\theta_1) = \int p(\theta_1, \theta_2)d\theta_2$. 
>* **Noise scale parameter**: when doing parameter estimation, it's a good idea to account for the possibility that our estimation for the error in our data might have been overestimated or underestimate (the latter could also be due to features in our data that the model cannot reproduce). We add this parameter as a scaling to the datapoint uncertainty. See LINK TO NOTEBOOK for details. 

More specifically about our bayesian model in the context of a magnetic star:

* For multiple observations of a given star, the value of $B_\mathrm{pole}$, $i$ and $\beta$ should remain the same from one observation to the next. We are therefore interested in the probability density of each of these parameters when taking in account all of the observations together. However, the rotational phase $\phi$ (and the noise scale parameter) can change from observation to observation. Furthermore, in most applications of `pyRaven`, the rotational period is not known, therefore the rotational phases cannot be determined from an already known ephemeris. 

    Therefore the strategy is to caluculate the joint likelihood $p(B_\mathrm{pole}, i, \beta, \phi)$. For each observations, we marginalize the likelihood probability over the rotational phases and noise scale parameter (taking into account the prior for these two parameters), and then combine the likelihood probabilities from each observations together. (See Petit&Wade 2012 for the mathematical description)

Ok, now let's talk about the files. In the list below, `[S]` stands for the Stokes parameter involved ("V" or "N1") and `[o]` stands for the observation index.

- **`lnLH_PARS_[S]_obs[o].h5`:** Contains the $p(\beta, B_\mathrm{pole}, \phi, i, \mathrm{noise})$ *likelihood* for parameter estimation. (Object can be read in with the <a href="../API/bayesobjects.html#pyRaven.BayesObjects.read_lnP_pars">`read_lnP_pars()`</a> class function)<br><br>

    :::{Tip}
    To obtain the posterior probability for a single observation, one could do: `lnLH.apply_all_prior(Jeff_B=...).normalize()`. The resulting 1D marginalization for all paramters can be plotted with the `.plot_mar()` function. If you'd like to a quick corner plot of Bpole, i, beta for that observation, you can do `.mar_phase_noise().plot_mar()`. 
    :::

- **`lnLH_PARS_[S]_mar_combined_flatprior.h5`:** Contains the $p(\beta, B_\mathrm{pole}, i)$ *likelihhood* probability the **combined observations**, each observation was **marginalzed for $\phi$ and $\mathrm{noise}$** with a **Flat** prior used for the noise parameter (Read in with <a href="../API/bayesobjects.html#pyRaven.BayesObjects.read_lnP_mar">`read_lnP_mar()`</a>)<br><br>


- **`lnLH_PARS_[S]_mar_combined_withprior.h5`:** Contains the $p(\beta, B_\mathrm{pole}, i)$ *likelihhood* probability the **combined observations**, each observation was **marginalzed for $\phi$ and $\mathrm{noise}$** with a **Jeffreys** prior used for the noise parameter (Read in with <a href="../API/bayesobjects.html#pyRaven.BayesObjects.read_lnP_mar">`read_lnP_mar()`</a>)<br><br>
    :::{tip}
    This file is usually the most important data product of the analysis. The other files are used for diagnostics. 
    :::


:::{dropdown} There is another set of files that is created to compute the odds-ratios. (expand for details)

NEED UPDATING!!!!

- **`lnLH_ODDS_[S]_obs[o].h5`:** Contains the $p(\beta, B_\mathrm{pole}, \phi, i)$ *likelihood* for the odds ratio calculation. This would be the same as a slice of `lnLH_PARS_[S]_obs[o].h5` that has a scale noise value of 1.0. (Object can be read in with the <a href="../API/bayesobjects.html#pyRaven.BayesObjects.read_lnP_odds">`read_lnP_odds()`</a> class function)<br><br>


- **`lnpost_ODDS_flatprior_[S]_obs[o].h5`:** Contains the $p(\beta, B_\mathrm{pole}, \phi, i)$ *un-normalized posterior* probability calculated **using flat priors** for all parameters. It is not normalized to 1.0, because the actual full marginalization value is necessary for the odds-ratio calculation. If this probability distribution is normalized (e.g. with the <a href=../API/bayesobjects.html#pyRaven.BayesObjects.lnP_odds.normalize>)`lnP_odds.normalize`</a> class function, then it would represent the *normalized* posterior probability for a parameter estimation done without the noise scale factor and with flat priors. (Read in with <a href="../API/bayesobjects.html#pyRaven.BayesObjects.read_lnP_odds"> `read_lnP_odds()`</a> function)<br><br>


- **`lnpost_ODDS_wprior_[S]_obs[o].h5`:** Contains the $p(\beta, B_\mathrm{pole}, \phi, i)$ **un-normalized posterior** probability for observation 'o' (with priors). If this probability distribution is normalized, it would represent the parameter estimation done without the noise scale and with the priors. (Read in with <a href="../API/bayesobjects.html#pyRaven.BayesObjects.read_lnP_odds">`read_lnP_odds()`</a>)<br><br>


- **`lnpost_ODDS_mar_flatprior_[S]_obs[o].h5`:** Contains the $p(\beta, B_\mathrm{pole}, i)$ *un-normalized posterior* probability **marginalzed for $\phi$**, for observation 'o', with **flat** priors (Read in with <a href="../API/bayesobjects.html#pyRaven.BayesObjects.read_lnP_mar">`read_lnP_mar()`</a>)<br><br>


- **`lnpost_ODDS_mar_wprior_[S]_obs[o].h5`:** Contains the $p(\beta, B_\mathrm{pole}, i)$ *un-normalized posterior* probability **marginalzed for $\phi$**, for observation 'o', with priors (Read in with <a href="../API/bayesobjects.html#pyRaven.BayesObjects.read_lnP_mar">`read_lnP_mar()`</a>)<br><br>

- **`lnpost_ODDS_mar_flatprior_[S].h5`:** Contains the $p(\beta, B_\mathrm{pole}, i) *un-normalized posterior* probability for the **combined observations**, but for which the priors are all **flat**) (Read in with <a href="../API/bayesobjects.html#pyRaven.BayesObjects.read_lnP_mar">`read_lnP_mar()`</a>)<br><br>

- **`lnpost_ODDS_mar_wprior_[S].h5`:** Contains the $p(\beta, B_\mathrm{pole}, i) *un-normalized posterior* probability for the **combined observations** (Read in with <a href="../API/bayesobjects.html#pyRaven.BayesObjects.read_lnP_mar">`read_lnP_mar()`</a>)

:::

THIS NEEDS TO MOVE TO THE NEXT NOTEBOOK!!!

The plot_summary function creates a PDF file with a variety of diagnostic graphs. 

The first series of graphs are the normalized and marginalized probability density for the (Bpole, beta, phi, incl, noise) parameters for each observations. 
* Pink curve is the 1D prior for each parameter.
* Solid lines are with priors, dashed lines are with flat priors 
* Black lines are with the scale noise parameter, and grey lines are witout the scale noise parameters. 
* The dashed lines are also a representation of the normalized likelihood. 

The second series of graphs show the corner plot for each observation with the phi and noise marginalized (for each observation, therefore the combination has not taken place yet). The probability densities are all normalized. Stokes V is the on left and the Null 1 profile is on the right. For each observations there are 4 consecutive pages:
* 1. flat priors and no scale noise (also represents the normalized LH)
* 2. applied priors and no scale noise
* 3. flat prios and with scale noise (also represents the normalized LH)
* 4. applied priors and with scale noise

Finally at the end, the last 4 corner plots are for the combined observations. Again, there are 4 pages in the same order as above. 

:::{tip}
These diagnostic graphs allows you to look at the impact of the priors, see the parameter correlations, and inspect the probability density for the nuisance parameters (phi and noise).
:::

The next steps is to extract commonly used results from these probabilities, mainly:
* The best fit parameters (MAP, MODE, etc) and an overplot with the observations
* The confidence regions for each parameters. 

(see LINK TO NOTEBOOK)